In [ ]:
from bs4 import BeautifulSoup as bs
from functools import reduce
from pandas.io.json import json_normalize
import json
import matplotlib as mpl
import pandas as pd
import re
import requests
import seaborn as sns
import unicodedata as uni
import os

In [ ]:
def read_json_to_dict(filename, path_to_file = None, encoding = 'utf-8'):
    
    path_to_file = reduce(lambda x, y: x if x is not None else y, [path_to_file, os.getcwd().replace('\\', '/')])
    
    if path_to_file[-1:] != '/':
        path_to_file = path_to_file + '/'
        
    full_file_path = path_to_file + '/' + filename
    
    with open(full_file_path, 'r', encoding = encoding) as file:
        data = json.load(file)
    
    return data
        

In [ ]:
linneburg_ratings = read_json_to_dict('linneburg.json', 'movie_jsons')

In [60]:
test = 'directors'
test[:-1]

'director'

In [ ]:
i = 0
while i < 5:
    print(linneburg_ratings['movies_rated'][0]['cast']['cast_list'][i]['actor_name'])
    i += 1

In [ ]:
for i in range(len(condensed_crew_list)):
    print(condensed_crew_list[i])

In [49]:
crew_list

[{'crew_role': 'directors',
  'number_assigned': 2,
  'crew_attributes': [{'name': 'Anthony Russo',
    'url': '/director/anthony-russo/'},
   {'name': 'Joe Russo', 'url': '/director/joe-russo/'}]},
 {'crew_role': 'producers',
  'number_assigned': 9,
  'crew_attributes': [{'name': 'Stan Lee', 'url': '/producer/stan-lee/'},
   {'name': 'Kevin Feige', 'url': '/producer/kevin-feige/'},
   {'name': 'James Gunn', 'url': '/producer/james-gunn/'},
   {'name': 'Jon Favreau', 'url': '/producer/jon-favreau/'},
   {'name': 'Michael Grillo', 'url': '/producer/michael-grillo/'},
   {'name': 'Victoria Alonso', 'url': '/producer/victoria-alonso/'},
   {'name': 'Jason Tamez', 'url': '/producer/jason-tamez/'},
   {'name': 'Lulu Morgan', 'url': '/producer/lulu-morgan/'},
   {'name': 'Trinh Tran', 'url': '/producer/trinh-tran/'}]},
 {'crew_role': 'writers',
  'number_assigned': 2,
  'crew_attributes': [{'name': 'Christopher Markus',
    'url': '/writer/christopher-markus/'},
   {'name': 'Stephen McFeely'

In [44]:
crew_wanted = ['director', 'producer', 'writer', 'composer']

condensed_crew_dict = {}
crew_list = linneburg_ratings['movies_rated'][32]['crew']['crew_list']

for i in range(len(crew_list)):
    for j in range(len(crew_wanted)):
        if crew_wanted[j] in crew_list[i]['crew_role']:
            condensed_crew_dict[crew_wanted[j]] = crew_list[i]['crew_attributes'][0]['name']
            print(crew_list[i]['crew_attributes'][0]['name'] + ': ' + crew_wanted[j] + ' ' + str(i))
            
condensed_crew_dict

Anthony Russo: director 0
Stan Lee: producer 1
Christopher Markus: writer 2
Alan Silvestri: composer 7


{'director': 'Anthony Russo',
 'producer': 'Stan Lee',
 'writer': 'Christopher Markus',
 'composer': 'Alan Silvestri'}

In [58]:
for i in range(len(crew_list)):
    for j in range(len(crew_wanted)):
        if crew_wanted[j] in crew_list[i]['crew_role']:
            print(crew_list[i]['crew_role'])
            print(crew_list[i]['crew_attributes'])

directors
[{'name': 'Anthony Russo', 'url': '/director/anthony-russo/'}, {'name': 'Joe Russo', 'url': '/director/joe-russo/'}]
producers
[{'name': 'Stan Lee', 'url': '/producer/stan-lee/'}, {'name': 'Kevin Feige', 'url': '/producer/kevin-feige/'}, {'name': 'James Gunn', 'url': '/producer/james-gunn/'}, {'name': 'Jon Favreau', 'url': '/producer/jon-favreau/'}, {'name': 'Michael Grillo', 'url': '/producer/michael-grillo/'}, {'name': 'Victoria Alonso', 'url': '/producer/victoria-alonso/'}, {'name': 'Jason Tamez', 'url': '/producer/jason-tamez/'}, {'name': 'Lulu Morgan', 'url': '/producer/lulu-morgan/'}, {'name': 'Trinh Tran', 'url': '/producer/trinh-tran/'}]
writers
[{'name': 'Christopher Markus', 'url': '/writer/christopher-markus/'}, {'name': 'Stephen McFeely', 'url': '/writer/stephen-mcfeely/'}]
composer
[{'name': 'Alan Silvestri', 'url': '/composer/alan-silvestri/'}]


In [ ]:
linneburg_ratings['movies_rated'][32]